In [ ]:
from setup_module import init_setup
from setup_module import import_data,save_and_load_list
import numpy as np
from classification_module import init_classification, combine_accuracy_graphs
import os
import matplotlib.pyplot as plt


In [ ]:
# samples, sample_comb, sample_iq, labels, snr = init_setup()


In [ ]:
# delete this cell when done debugging XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
if not os.path.exists("data"):
    # Create the folder if it doesn't exist
    os.makedirs("data")

if not os.listdir("data"):
    filepath = "/home/bendegani/AMC_v2/GOLD_XYZ_OSC.0001_1024.hdf5"
    import_data(filepath)

idx = 2
print("loading data")
# load the data
cumulants = save_and_load_list(2, "cumulants_vec", idx)
# cum_dx_vec = save_and_load_list(2, "cum_dx_vec", idx)
cum_real_vec = save_and_load_list(2, "cum_real_vec", idx)
cum_imag_vec = save_and_load_list(2, "cum_imag_vec", idx)
modulation = save_and_load_list(2, "labels", idx)
snr_vec = save_and_load_list(2, "snr_vec", idx)


In [ ]:
# delete this cell when done debugging XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
# c_20 # c_21  # c_40  # c_41 # c_42  # c_60 # c_63 # c_80

def cumulant_fix_complex(cumulants):
    # Gets the features vector from the cumulants
    features_vec = []
    for ii in range(len(cumulants)):
        cum_abs = (abs(cumulants[ii]))
        # initiate output
        feats = []
        # for jj in range(len(cum_abs)):
        #     feats.append((cum_abs[jj]))
        # feats.append((cum_abs[0])/cum_abs[1])
        feats.append(pow((cum_abs[0]),2)/cum_abs[2])
        feats.append(pow((cum_abs[0]),2)/cum_abs[3])
        feats.append(pow((cum_abs[1]),3)/cum_abs[5])
        feats.append(pow((cum_abs[1]),4)/cum_abs[7])
        feats.append(pow((cum_abs[6]),4)/pow(cum_abs[7],3))
        feats.append(pow((cum_abs[2]),3)/pow(cum_abs[5],2))
        feats.append(np.log(cum_abs[0]))
        feats.append(np.log(cum_abs[1]))
        feats.append(np.log(cum_abs[4]))
        features_vec.append(feats)
    return features_vec

def cumulant_fix_iq(cum_real, cum_imag):
    # Gets the features vector from the cumulants
    features_vec = []
    for ii in range(len(cum_real)):
        real_abs = (abs(cum_real[ii]))
        imag_abs = (abs(cum_imag[ii]))
        # initiate output
        feats = []
        # for jj in range(len(cum_abs)):
        #     feats.append((cum_abs[jj]))
        
        # feats.append((cum_abs[0])/cum_abs[1])
        feats.append(pow((cum_real[0]),2)/cum_real[1])
        feats.append(pow((cum_real[0]),3)/cum_real[2])
        feats.append(pow((cum_real[0]),4)/cum_real[3])
        feats.append(pow((cum_real[1]),2)/cum_real[3])
        feats.append(pow((cum_real[2]),4)/pow(cum_real[3],3))
        
        feats.append(pow((cum_imag[0]),2)/cum_imag[1])
        feats.append(pow((cum_imag[0]),3)/cum_imag[2])
        feats.append(pow((cum_imag[0]),4)/cum_imag[3])
        feats.append(pow((cum_imag[1]),2)/cum_imag[3])
        feats.append(pow((cum_imag[2]),4)/pow(cum_imag[3],3))
        features_vec.append(feats)
    return features_vec

try:
    print("fixing for absolute value")
    cumulants_vec = cumulant_fix_complex(cumulants)
    print('fixed cumulants')
    # cum_dx_vec = cumulant_fix_complex(cum_dx_vec)
    print('fixed cumulants dx')
    cum_iq_vec = cumulant_fix_iq(cum_real_vec, cum_imag_vec)
    print('fixed cumulants IQ')
except:
    print('problem')
    
# convert to array
print("convert data")
labels = np.array(modulation)
snr = np.array(snr_vec)
samples = np.array(cumulants_vec)
# samples_dx = np.array(cum_dx_vec)
sample_iq = np.array(cum_iq_vec)
# sample_comb = np.concatenate((samples, samples_dx), 1)
sample_comb = np.concatenate((samples, sample_iq), 1)

In [ ]:
easy_mode = 1
group1 = [
        "OOK",
        "AM-SSB-WC",
        "AM-SSB-SC",
        "AM-DSB-WC",
        "AM-DSB-SC",
        "FM",
        "GMSK",
        "OQPSK",
    ]  # Low Order and Analog
group2 = [
        "4ASK",
        "8ASK",
        "16QAM",
        "32QAM",
        "64QAM",
        "128QAM",
        "256QAM",
    ]  # ASK and QAM
group3 = [
        "BPSK",
        "QPSK",
        "8PSK",
        "16PSK",
        "32PSK",
        "16APSK",
        "32APSK",
        "64APSK",
        "128APSK",
    ]  # PSK and APSK

if easy_mode == 1:
    easy_mods = [
        "OOK",
        "4ASK",
        "BPSK",
        "QPSK",
        "8PSK",
        "16QAM",
        "AM-SSB-SC",
        "AM-DSB-SC",
        "FM",
        "GMSK",
        "OQPSK",
    ]
    easy_mask = np.isin(labels, easy_mods)
    labels = labels[easy_mask]
    snr = snr[easy_mask]
    samples = samples[easy_mask]
    # sample_comb = sample_comb[easy_mask]
    # sample_iq = sample_iq[easy_mask]
    easy_mask = np.isin(labels, easy_mods)
    labels = labels[easy_mask]
    snr = snr[easy_mask]
    

    group1 = np.array(group1)[np.isin(group1,easy_mods)]
    group2 = np.array(group2)[np.isin(group2,easy_mods)]
    group3 = np.array(group3)[np.isin(group3,easy_mods)]
    
groups = [group1, group2, group3]

In [ ]:

print("we are here")
print("classification for cumulants...")
accuracy_data1 = init_classification(samples, labels, snr, "cumulants", groups)
print("classification for cumulants+dx...")
accuracy_data2 = init_classification(sample_comb, labels, snr, "cumulants+dx", groups)
print("classification for cumulants IQ...")
accuracy_data3 = init_classification(sample_iq, labels, snr, "cumulants_iq", groups)


In [ ]:
combine_accuracy_graphs(accuracy_data1, accuracy_data2, accuracy_data3)